# 创建可管理的属性
- 在对实例属性的获取和设定上，我们希望增加一些额外的处理过程（比如类型检查或者验证）

## 要自定义对属性的访问，一种简单的方式是将其定义为property

In [1]:
class Person:
    def __init__(self, first_name):
        self.first_name = first_name
        
    # Getter function
    @property
    def first_name(self):
        return self._first_name
    
    # Setter function
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Excepted a string')
        self._first_name = value
    
    # Deleter function (optional)
    @first_name.deleter
    def first_name(self):
        raise AttributeError("Cannot delete attributes")

上述代码中，一共有三个互相关联的方法，他们必须有着相同的名称。第一个方法是一个getter函数，并且将first_name定义为了property属性。其他两个方法将可选的setter和deleter函数附加到了first_name属性上。需要重点强调的是，<span class="mark">除非first_name已经通过@property的方式定义为了porperty属性</span>，否则不能定义@first_name.setter和@first_name.deleter装饰器的。

## property的重要特性就是它看起来像一个普通的属性，但是根据访问它的方式的不同，会自动触发getter、setter以及deleter方法。

In [2]:
a = Person('Jack')
a.first_name  # call the getter

'Jack'

In [3]:
a.first_name = 42  # call the setter

TypeError: Excepted a string

In [4]:
del a.first_name  # call the deleter

AttributeError: Cannot delete attributes

- 当我们实现一个property时，底层的数据仍需要被保存到某个地方。因此在get和set方法中，可以看到我们是直接对_first_name进行操作的，就是数据实际保存的地方。
- 在```__init__()```方法中设置的是self.first_name是因为我们想在类进行初始化的时候就进行赋值的类型检查，因为self.first_name = a 会触发@first_name.setter，因此也会跳过上面的赋值语句，而去直接赋值给self._first_name

## 讨论

### property属性实际上就是把一系列的方法绑定到一起。如果检查类的property属性就会发现property自身所持有的属性fget、fset、fdel所代表的原始方法

In [5]:
Person.first_name.fget

<function __main__.Person.first_name>

In [6]:
Person.first_name.fset

<function __main__.Person.first_name>

In [7]:
Person.first_name.fdel

<function __main__.Person.first_name>

- 一般并不会直接去调用fget或者fset,但是当我们访问property属性时会自动触发对这些方法的调用
- 只有当确实需要在访问属性时完成一些额外的处理任务时，才应该使用property。
- 有时候java程序员会觉得所有的访问都需要通过getter和setter处理，那么他们代码应该这样：

In [8]:
class Person:
    def __init__(self, first_name):
        self.first_name = first_name

    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        self._first_name = value

### 如果property并不会完成额外的处理任务，那么就不需要把代码写成上面这个样子。

- 第一，这么做会使得代码变得更加啰嗦，对其他人来说也比较困惑。
- 第二，这么做会让程序变慢很多
- 最后，这么做不会给设计带来真正的好处

 ### property也可以用来定义需要计算的属性，这类属性并不会实际保存起来，而是根据需要完成计算。

In [9]:
import math
class Circle:
    def __init__(self, radius):
        self.radius = radius
        
    @property
    def area(self):
        return math.pi * self.radius ** 2
    
    @property
    def perimeter(self):
        return 2 * math.pi * self.radius

这里对property的使用使得实例的接口变得非常统一，radius、areas以及perimeter都能够简单地以属性的形式进行访问，而不必将属性和方法调用混在一起使用了。

In [10]:
c = Circle(4.0)
c.radius

4.0

<span class="mark">不需要使用()</span>

In [11]:
c.area

50.26548245743669

In [12]:
c.perimeter

25.132741228718345

### 不要编写那种定义了大量重复性property的代码。重复的代码会导致膨胀，容易出错，而且代码也十分丑陋。

事实证明，利用描述符或者闭包能够更好地完成同样的任务，具体参见8.9节和9.21节